# Esercizi Lezione 7 - Con Soluzioni

## Esercizio 1
**Obiettivo:** Costruire un modello di regressione polinomiale che approssima l’andamento del numero di passeggeri nel tempo.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime

### Punto 1: Caricamento e preparazione del dataset

In [ ]:
data_df = pd.read_csv("data.csv")
data_df["date"] = pd.to_datetime(data_df["date"], format='%Y-%m').dt.to_period('M')
data_df.dropna(axis=0, inplace=True)
data_df.reset_index(inplace=True)
data_df

### Punto 2: Aggiunta colonna `mese_numerico`

In [ ]:
data_df["mese_numerico"] = range(len(data_df))
data_df

### Punto 3: Divisione train/test e regressione

In [ ]:
X = data_df["mese_numerico"].values.reshape(-1, 1)
y = data_df["passengers"].values

# Visualizzazione
plt.figure(figsize=(12,6))
plt.plot(X, y, marker='o', linestyle='-', label='Numero passeggeri')
plt.title('Numeri di passeggeri per mese')
plt.xlabel('Date')
plt.ylabel('Passengers')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Divisione train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=40, shuffle=False)

# Regressione lineare
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)
y_pred_linear = model_linear.predict(X_test)

# Regressione polinomiale di grado 3
model_poly3 = make_pipeline(PolynomialFeatures(3), LinearRegression())
model_poly3.fit(X_train, y_train)
y_pred_poly3 = model_poly3.predict(X_test)

### Punto 4: Calcolo MAE e RMSE

In [ ]:
mae_linear = mean_absolute_error(y_test, y_pred_linear)
mae_poly3 = mean_absolute_error(y_test, y_pred_poly3)
rmse_linear = mean_squared_error(y_test, y_pred_linear, squared=False)
rmse_poly3 = mean_squared_error(y_test, y_pred_poly3, squared=False)

### Punto 5: Visualizzazione Plotly

In [ ]:
df_plot = pd.DataFrame({
    "mese_numerico": X_test.ravel(),
    "Reale": y_test,
    "Lineare": y_pred_linear,
    "Polinomiale grado 3": y_pred_poly3
})

df_long = df_plot.melt(id_vars="mese_numerico", 
                       value_vars=["Reale", "Lineare", "Polinomiale grado 3"], 
                       var_name="Serie", value_name="Passeggeri")

fig = px.line(df_long,
              x="mese_numerico",
              y="Passeggeri",
              color="Serie",
              marker=True,
              title="Confronto valori reali vs stime (lineare & polinomiale grado 3)")

fig.update_layout(xaxis_title="Mese numerico",
                  yaxis_title="Passengers",
                  template="plotly_white")

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.02, y=0.98, showarrow=False, align="left",
    text=f"""MAE lineare = {mae_linear:.2f}<br>
           MAE polinomiale = {mae_poly3:.2f}<br>
           RMSE lineare = {rmse_linear:.2f}<br>
           RMSE polinomiale = {rmse_poly3:.2f}""")

fig.show()

## Esercizio 2: Web App interattiva con Dash

In [ ]:
x = np.random.uniform(-3, 3, 100).reshape(-1, 1)
y_true = x.ravel()**3 - x.ravel()
y = y_true + np.random.normal(scale=1.0, size=len(x))

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=False
)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3("Seleziona il grado del polinomio"),
    dcc.Slider(id='slider-numero', min=1, max=10, step=1, value=1,
               marks={i: str(i) for i in range(1, 11)}),
    html.Div(id='output-plot')
])

@app.callback(
    Output('output-plot', 'children'),
    Input('slider-numero', 'value')
)
def plot_with_regression(valore):
    model_polyN = make_pipeline(PolynomialFeatures(valore), LinearRegression())
    model_polyN.fit(x_train, y_train)
    x_grid = np.linspace(x.min(), x.max(), 200).reshape(-1, 1)
    y_grid = model_polyN.predict(x_grid)

    df_real = pd.DataFrame({"x": x.ravel(), "y": y})
    df_curve = pd.DataFrame({"x": x_grid.ravel(), "y": y_grid})

    fig = px.scatter(df_real, x="x", y="y",
                     title="Dynamic regression model",
                     labels={"x": "x value", "y": "y value"},
                     opacity=0.7)

    fig.add_traces(px.line(df_curve, x="x", y="y").update_traces(name=f"Poly degree {valore}").data)
    fig.update_layout(template="plotly_white")
    return dcc.Graph(figure=fig)

if __name__ == "__main__":
    app.run_server(debug=True)